# Image Classification: More Pets


This notebook trains a model using TensorFlow to classify an image as a rabbit, mouse, hamster, fish, lizard, or snake. 

Below we do the following:

1. Setup training environment.
2. Load images of rabbits, mic, hamsters, fish, lizards, and snakes.
3. Train an image classifier model using the TensorFlow library, leveraging transfer learning and MobileNetV2.
4. Convert the model from TensorFlow to TF-Lite. 

This workflow is based on two examples provided in the TensorFlow-Lite documentation: 

* https://www.tensorflow.org/tutorials/images/transfer_learning
* https://github.com/tensorflow/examples/blob/master/community/en/flowers_tf_lite.ipynb

It is also important to note that this Notebook is using a GPU run-time to speed up the training process. Without a GPU, model training will take significantly longer! 

In [0]:
# First, we need to import the necessary libraries.
!pip install tensorflow-gpu==2.0.0-beta1 

import tensorflow as tf

import os
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# Let's validate our version of TensorFlow
tf.__version__

## Data Preparation

The training data for this example are hundreds of images of various animals, pulled from the [Open Images Dataset v4](https://storage.googleapis.com/openimages/web/download_v4.html).

The code below does the following: 

* Downloads the data from a public S3 bucket provided by Skafos. 
* Validates that the expected categories are present
* Creates the necessary `ImageDataGenerator` required by TensorFlow and then splits the data into training and validation sets
* Saves the category labels in a file `labels.txt` for later download. 

In [0]:
# Specify the location of the data set
_URL = "https://s3.amazonaws.com/skafos.example.data/ImageClassifier/MorePets.tar.gz"

# Download the file and extract the images 
zip_file = tf.keras.utils.get_file(origin=_URL, 
                                   fname="MorePets.tar.gz", 
                                   extract=True)

# Specify the base directory for the data set
base_dir = os.path.join(os.path.dirname(zip_file), 'MorePets')

# Validate that the categories present are what you'd expect
os.listdir(base_dir)

In [0]:
# Set the image side and batch size. Becasue we will be using MobileNetV2, we will set the image size to 224x224. 
# For more information, see: https://keras.io/applications/#mobilenetv2
IMAGE_SIZE = 224  
BATCH_SIZE = 64
NUMBER_OF_LABELS = len(os.listdir(base_dir)) 

# Create data generator with appropriate scaling and train/validation split
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

# Create training dataset
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

# Create validation dataset
val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

In [0]:
# Create labels.txt file, which we will need to export and save for use in our app
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

## Model Training

Transfer learning leverages pre-existing "base models" to make your model training faster. In this example, we will use the MobileNetV2 model as our base model. This model was developed at Google, and pre-trained on the ImageNet dataset, a large dataset of 1.4M images and 1000 classes of web images.

To construct a model leveraging transfer learning in TensorFlow, we will take the following steps: 

* Do feature extraction by freezing the convolutional base model and adding classification layers on top. 
* Compile + fit the model
* Fine tune the model by unfreezing the base model, and allowing fine-tuning of the base model weights for additional layers of the network. 

For more information about transfer learning in TensorFlow, [please review the TensorFlow documentation](https://www.tensorflow.org/tutorials/images/transfer_learning). Thie documentation will provide more context and information about how transfer learning works in TensorFlow. 

In [0]:
# Set the image shape based on the image size specified above. 
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3) 

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

# Freeze the base_model
base_model.trainable = False

# Add classification layers on top of the base model
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(NUMBER_OF_LABELS, activation='softmax')
])

# Compile the model. The optimal loss function depends on the type of model you want to build. 
# This article provides a thorough summary: https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# To get a summary of the model, uncomment the line below
#model.summary()

In [0]:
# Train the model. This will likely take a few minutes. Increasing the number of epochs will increase the model training time, and can be tuned for optimal accuracy.  
history = model.fit(train_generator, 
                    epochs=10, 
                    validation_data=val_generator)

In [0]:
# Begin the fine-tuning process by allowing training of the base_model
base_model.trainable = True

# This print statement will tell us the number of layers in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine tune from this layer onwards. This number must be less than len(base_model.layers)). In this example, we've chosen 100. 
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False
  
# Compile the model. Note that we have chosen a different optimizer from previously
model.compile(optimizer = tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
# Train the model, fine tuning this time. 
history_fine = model.fit(train_generator, 
                         epochs=20,
                         validation_data=val_generator)

## Model Validation

Using the matplot lib library, we can plot our model accuracy and loss. 

We can also use the model to do inference on a new image as another way to test it. 

In [0]:
# Plotting accuracy and loss
acc = history_fine.history['accuracy']
val_acc = history_fine.history['val_accuracy']

loss = history_fine.history['loss']
val_loss = history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [0]:
# Let's use this model to make a prediction on a new image to test it
import numpy as np
import PIL.Image as Image

# The image URL below is from the Google Open Images Dataset
test_image_URL = 'https://c1.staticflickr.com/4/3581/3407856156_415b3fd8ee_o.jpg'
test_image = tf.keras.utils.get_file('image.jpg', test_image_URL) 
test_image = Image.open(test_image)

# Keep aspect ratio when resizing 
#wpercent = (IMAGE_SIZE / float(test_image.size[0]))
#hsize = int((float(test_image.size[1]) * float(wpercent)))
test_image = test_image.resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)

# Let's view the image to see what it is
test_image

In [0]:
# Now we need to scale the model before predicting and add a batch size via the tf.expand_dims command
test_image = np.array(test_image)/255.0
test_image.shape = tf.shape(tf.expand_dims(test_image, 0)) 

# Use the model object to predict what this image is
prediction = model.predict(test_image)

# Get the predicted class label as the max value from the array of predictions
predicted_class = np.argmax(prediction[0], axis=-1)

# Get the corresponding label from train_generator.class_indices
predicted_class = np.argmax(prediction[0], axis=-1)

## Convert your model to TensorFlow Lite and Export

As a final step to optimize your model for use on mobile devices, we need to convert our model from TensorFlow to TensorFlow-Lite. After conversion, we will download it for inclusion in our mobile app. 

More information about converting from TensorFlow to TensorFlow Lite is available here: https://www.tensorflow.org/lite/r2/convert/python_api

In [0]:
# Convert our model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [0]:
# Download the model for inclusion in a mobile app. 
# Soon, we will be able to use the Skafos SDK to deliver the model directly to Skafos 

from google.colab import files

files.download('model.tflite')
files.download('labels.txt')